# Использование доступных моделей
Данный ноутбук показывает базовый порядок работы с библиотекой.
- загрузка датасета
- деление на трейн и тест
- обучение модели
- сравнение с бейзлайном

Документацию можно собрать из папки `docs` командой `make html`. Она создаст папку `_build` с документацией -- `sponge-bob-magic/docs/_build/html/index.html`

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Целевая архитектура библиотеки - ЛД 3.0, поэтому используем Spark.
Для отладки моделей (не на кластере) создаём локальную сессию.

Объект `State` позволяет использовать одну и ту же сессию Spark в разных объектах.
Если сессия уже инициализированна, то её нужно передать при инициализации `State`. Модули библиотеки, которым необходимо использовать спарк-сессию, например для конвертации из пандас в спарк, будут искать сессию именно в `State`.

По умолчанию создастся дефолтная сессия. Простой способ получить сессию с заданным количеством выделенной памяти -- функция `get_spark_session` модуля `session_handler`.

In [3]:
from sponge_bob_magic.session_handler import State

spark = State().session
spark

## Подготовка данных <a name='data-preparator'></a>
Библиотека содержит утилиты для скачивания и парсинга популярных датасетов для рекомендаций.
Если датасет не найден в заданной директории, он будет автоматически скачан и обработан.

Данные датасета доступны в виде `pandas.DataFrame` атрибутов объекта.
Посмотреть доступные данные можно с помощью метода `info`.

In [4]:
from sponge_bob_magic.datasets import MovieLens

data = MovieLens("1m")
data.info()

ratings


,user_id,item_id,relevance,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968



users


,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117



items


,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


Внутренний формат данных в либе -- спарк датафрейм. Сплиттеры умеют втоматически конвертировать пандас датафреймы, но модели сейчас ожидают именно спарк, причем с заданными обязательными колонками: `user_id, item_id, timestamp, relevance, context`. 

Получить данные в нужном формате можно с помощью `DataPreparator.transform_log`, который создаст нужные колонки, если их нет.

In [5]:
from sponge_bob_magic.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance",
        "timestamp": "timestamp"
    }
)

Библиотека содержит различные схемы валидации рекомендательных систем, встречающиеся в литературе.

`UserSplitter` отбирает для теста некоторое количество или долю объектов для каждого пользователя.

In [6]:
from sponge_bob_magic.splitters import UserSplitter

splitter = UserSplitter(
    drop_cold_items=True,
    drop_cold_users=True,
    item_test_size=1,
    user_test_size=500,
    seed=1234,
    shuffle=True
)
train, test = splitter.split(log)
(
    train.count(), 
    test.count()
)

(999709, 500)

## NMF
Простейший пример использования DL в рекомендациях

In [7]:
from sponge_bob_magic.models import NeuroMFRec

nmf = NeuroMFRec(
    learning_rate=0.01,
    epochs=10,
    embedding_dimension=100
)

/Users/alexxl/sponge-bob-magic/venv/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [8]:
%%time
nmf.fit(log=train)

CPU times: user 3min 5s, sys: 7.43 s, total: 3min 12s
Wall time: 4min 55s


In [12]:
%%time

recs = nmf.predict(
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    log=train,
    filter_seen_items=True
)

CPU times: user 2.14 s, sys: 210 ms, total: 2.35 s
Wall time: 37.3 s


В библиотеке реализованы различные метрики качества рекомендательных систем, встречающихся в литературе.
Их можно использовать напрямую, либо запоминать результаты с помощью класса `Experiment`.

In [13]:
from sponge_bob_magic.metrics import HitRate, NDCG
from sponge_bob_magic.experiment import Experiment

metrics = Experiment(test, {NDCG(): 10, 
                            HitRate(): 10})

metrics.add_result("NMF", recs)
metrics.pandas_df.loc["NMF"]

NDCG@10       0.019911
HitRate@10    0.032000
Name: NMF, dtype: float64

## SLIM
Один из простых, но эффективных алгоритмов 

In [14]:
from sponge_bob_magic.models import SlimRec

slim = SlimRec(lambda_=0.01, beta=0.3)

In [15]:
%%time

slim.fit(log=train)

CPU times: user 5.55 s, sys: 331 ms, total: 5.88 s
Wall time: 9.42 s


In [16]:
%%time

recs = slim.predict(
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    log=train,
    filter_seen_items=True
)

CPU times: user 36.8 ms, sys: 8.29 ms, total: 45.1 ms
Wall time: 8.21 s


In [17]:
metrics.add_result("SLIM", recs)
metrics.pandas_df.loc["SLIM"]

NDCG@10       0.178872
HitRate@10    0.334000
Name: SLIM, dtype: float64

## ALS
Библиотека также содержит классические алгоритмы рекомендаций, например, матричную факторизацию

In [18]:
from sponge_bob_magic.models import ALSRec

als = ALSRec(rank=100)

In [19]:
%%time
als.fit(log=train)

CPU times: user 60.6 ms, sys: 9.12 ms, total: 69.7 ms
Wall time: 40.6 s


In [20]:
%%time
recs = als.predict(
    k=10,
    users=test.select('user_id').distinct(),
    items=test.select('item_id').distinct(),
    log=train,
    filter_seen_items=True
)

CPU times: user 29.4 ms, sys: 7.12 ms, total: 36.5 ms
Wall time: 7.97 s


In [21]:
metrics.add_result("ALS", recs)
metrics.pandas_df.loc["ALS"]

NDCG@10       0.057582
HitRate@10    0.140000
Name: ALS, dtype: float64

## Своя модель
Для построения своей модели нужно использовать тот же самый split, что и для бейзлайнов

In [22]:
train.toPandas().to_csv("train.csv", index=False)

/Users/alexxl/sponge-bob-magic/venv/lib/python3.7/site-packages/pyarrow/__init__.py:157: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
  warnings.warn("pyarrow.open_stream is deprecated, please use "


In [23]:
!head train.csv

user_id,item_id,relevance,timestamp
1090,1961,4.0,2000-11-23 01:20:08
1090,2428,2.0,2000-11-23 01:27:11
1090,1208,3.0,2000-11-23 01:21:17
1090,296,5.0,2000-11-23 01:06:15
1090,318,5.0,2000-11-23 01:11:51
1090,1704,4.0,2000-11-23 01:12:48
1090,2433,3.0,2000-11-23 01:26:01
1090,1569,4.0,2000-11-23 01:02:22
1090,110,3.0,2000-11-23 01:12:48


Здесь магия: можно взять train и обучить на нём свою любимую модель вне библиотеки

Также нужно выдать рекомендации обученной моделью

In [24]:
recs.toPandas().to_csv("recs.csv", index=False)

Теперь нужно прочитать рекомендации в формате, поддерживаемом библиотекой

In [25]:
recs = DataPreparator().transform(
    path="recs.csv",
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "relevance"
    },
    header=True,
    format_type="csv"
)

и сравнить качество своей модели с бейзлайнамии

In [26]:
metrics.add_result("my_model", recs)
metrics.pandas_df

,NDCG@10,HitRate@10
NMF,0.019911,0.032
SLIM,0.178872,0.334
ALS,0.057582,0.140
my_model,0.057582,0.140
